In [23]:
import pandas as pd
import sys
import csv

In [24]:
results = pd.read_csv("Final_RQ_results_Median_Delay.csv")

In [25]:
results

,Unnamed: 0,project,method,algorithm,batch_size,builds_reqd,batch_delays,sbs_delays
0,0,cloud_controller_ng.csv,gr,BATCHBISECT,2,76.284585,1.0,0.0
1,1,cloud_controller_ng.csv,gr,BATCHBISECT,4,76.679842,2.0,0.0
2,2,cloud_controller_ng.csv,gr,BATCH4,4,75.889328,2.0,0.0
3,3,cloud_controller_ng.csv,gr,BATCHSTOP4,4,75.889328,2.0,0.0
4,4,cloud_controller_ng.csv,gr,BATCHBISECT,8,82.213439,4.0,0.0
...,...,...,...,...,...,...,...,...
622,622,rails.csv,ml,BATCHSTOP4,16,66.892349,7.0,1.0
623,623,rails.csv,ml,BATCHBISECT,2,60.257278,0.0,1.0
624,624,rails.csv,ml,BATCHBISECT,4,60.392688,1.0,1.0
625,625,rails.csv,ml,BATCHBISECT,8,64.251862,3.0,2.0


In [26]:
results = results.drop('Unnamed: 0', axis='columns')

In [27]:
batch_delays = results['batch_delays'].tolist()
sbs_delays = results['sbs_delays'].tolist()
total_delays = []

In [28]:
for count in range(len(batch_delays)):
    total_delays.append( batch_delays[count] + sbs_delays[count])

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [17]:
results['total_delay'] = total_delays

In [18]:
results

,project,method,algorithm,batch_size,builds_reqd,batch_delays,sbs_delays,total_delay
0,cloud_controller_ng.csv,gr,BATCHBISECT,2,76.284585,127.0,0.0,127.0
1,cloud_controller_ng.csv,gr,BATCHBISECT,4,76.679842,381.0,0.0,381.0
2,cloud_controller_ng.csv,gr,BATCH4,4,75.889328,381.0,0.0,381.0
3,cloud_controller_ng.csv,gr,BATCHSTOP4,4,75.889328,381.0,0.0,381.0
4,cloud_controller_ng.csv,gr,BATCHBISECT,8,82.213439,893.0,0.0,893.0
...,...,...,...,...,...,...,...,...
622,cloud_controller_ng.csv,ml,BATCHSTOP4,16,86.166008,1920.0,0.0,1920.0
623,cloud_controller_ng.csv,ml,BATCHBISECT,2,62.450593,106.0,11.0,117.0
624,cloud_controller_ng.csv,ml,BATCHBISECT,4,71.936759,360.0,3.0,363.0
625,cloud_controller_ng.csv,ml,BATCHBISECT,8,79.446640,868.0,3.0,871.0


In [21]:
method = ['ml', 'ssr', 'gr']
algorithms = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
batchsize = [2, 4, 8, 16]
project_list = list(set(results['project'].tolist()))

In [22]:
for m in method:
    
    filename = m + '_pivot_median_delay.csv'
    file = open(filename, 'w')
    res_headers = ['project', 'NB_builds', 'NB_delay', 'NB_Batch', 'B4_builds', 'B4_delay', 'B4_Batch', 'BB_builds', 'BB_delay', 'BB_Batch', 'BS4_builds', 'BS4_delay', 'BS4_Batch']
    res_writer = csv.writer(file)
    res_writer.writerow(res_headers)
    
    
    for p in project_list:
        
        nb_builds = 0
        nb_delay = 0
        nb_batch = 0
        
        b4_builds = 0
        b4_delay = 0
        b4_batch = 0
        
        bs4_builds = 0
        bs4_delay = 0
        bs4_batch = 0
        
        bb_builds = 0
        bb_delay = 0
        bb_batch = 0
        
        for alg in algorithms:
            data = results[ (results['method'] == m) & (results['algorithm'] == alg) & (results['project'] == p)]
            data = data.drop(data[data.batch_size < 2].index)
            data = data.drop(data[data.batch_size > 16].index)

            best_batch = 0
            lowest_distance = sys.maxsize
            delays = data['total_delay'].tolist()
            builds_reqd = data['builds_reqd'].tolist()
            batch_size = data['batch_size'].tolist()

            for i in range(len(delays)):
                d = (builds_reqd[i]**2 + delays[i]**2)**0.5
                if d < lowest_distance:
                    
                    best_batch = batch_size[i]
                    lowest_distance = d
                    
                    if alg == 'NOBATCH':
                        nb_builds = builds_reqd[i]
                        nb_delay = delays[i]
                        nb_batch = best_batch
                    elif alg == 'BATCHBISECT':
                        bb_builds = builds_reqd[i]
                        bb_delay = delays[i]
                        bb_batch = best_batch
                    elif alg == 'BATCHSTOP4':
                        bs4_builds = builds_reqd[i]
                        bs4_delay = delays[i]
                        bs4_batch = best_batch
                    else:
                        b4_builds = builds_reqd[i]
                        b4_delay = delays[i]
                        b4_batch = best_batch
            print('{} - {} - {} - {}'.format(p, m, alg, best_batch))
        res_writer.writerow([p, nb_builds, nb_delay, nb_batch, b4_builds, b4_delay, b4_batch, bb_builds, bb_delay, bb_batch, bs4_builds, bs4_delay, bs4_batch])
        

loomio.csv - gr - BATCH4 - 4
loomio.csv - gr - BATCHBISECT - 2
loomio.csv - gr - BATCHSTOP4 - 4
graylog2-server.csv - gr - BATCH4 - 4
graylog2-server.csv - gr - BATCHBISECT - 2
graylog2-server.csv - gr - BATCHSTOP4 - 4
gradle.csv - gr - BATCH4 - 4
gradle.csv - gr - BATCHBISECT - 2
gradle.csv - gr - BATCHSTOP4 - 4
open-build-service.csv - gr - BATCH4 - 4
open-build-service.csv - gr - BATCHBISECT - 2
open-build-service.csv - gr - BATCHSTOP4 - 4
vagrant.csv - gr - BATCH4 - 4
vagrant.csv - gr - BATCHBISECT - 2
vagrant.csv - gr - BATCHSTOP4 - 4
metasploit-framework.csv - gr - BATCH4 - 4
metasploit-framework.csv - gr - BATCHBISECT - 2
metasploit-framework.csv - gr - BATCHSTOP4 - 4
rubinius.csv - gr - BATCH4 - 4
rubinius.csv - gr - BATCHBISECT - 2
rubinius.csv - gr - BATCHSTOP4 - 4
sonarqube.csv - gr - BATCH4 - 4
sonarqube.csv - gr - BATCHBISECT - 2
sonarqube.csv - gr - BATCHSTOP4 - 4
concerto.csv - gr - BATCH4 - 4
concerto.csv - gr - BATCHBISECT - 2
concerto.csv - gr - BATCHSTOP4 - 4
jruby.c

In [45]:
batch_reps = {0: 'na', 2: '^*^*', 4: '^\#', 6: '^\%', 8:'^\#^\#', 16:'^\wedge'}

In [46]:
print(batch_reps[2])

^*^*


In [47]:
ssr = pd.read_csv('gr_pivot_result_old.csv')

In [48]:
projects = ssr['project'].tolist()
col1 = ssr['NB_builds'].tolist()
col2 = ssr['NB_delay'].tolist()
nb_batch = ssr['NB_Batch'].tolist()
col3 = ssr['B4_builds'].tolist()
col4 = ssr['B4_delay'].tolist()
b4_batch = ssr['B4_Batch'].tolist()
col5 = ssr['BB_builds'].tolist()
col6 = ssr['BB_delay'].tolist()
bb_batch = ssr['BB_Batch'].tolist()
col7 = ssr['BS4_builds'].tolist()
col8 = ssr['BS4_delay'].tolist()
bs4_batch = ssr['BS4_Batch'].tolist()

In [27]:
print(batch_reps[nb_batch[3]])

na


In [28]:
for i in range(len(projects)):
    print("{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} \\\\".format(projects[i],  
                                                                round(col3[i], 3), batch_reps[b4_batch[i]], round(col4[i], 3), batch_reps[b4_batch[i]], 
                                                                round(col5[i], 3), batch_reps[bb_batch[i]], round(col6[i], 3), batch_reps[bb_batch[i]], 
                                                                round(col7[i], 3), batch_reps[bs4_batch[i]], round(col8[i], 3), batch_reps[bs4_batch[i]] ))
    print("\hline")

nan & 0.0na & 0.0na & 0.0na & 0.0na & 0.0na & 0.0na \\
\hline
cloud_controller_ng.csv & 36.364^\# & 287.0^\# & 42.292^*^* & 113.0^*^* & 36.364^\# & 287.0^\# \\
\hline
orbeon-forms.csv & 13.305^\# & 151.0^\# & 21.03^*^* & 74.0^*^* & 13.305^\# & 151.0^\# \\
\hline
sonarqube.csv & 18.333^\# & 249.0^\# & 25.0^*^* & 100.0^*^* & 18.333^\# & 249.0^\# \\
\hline
jruby.csv & 48.426^\# & 494.0^\# & 53.753^*^* & 193.0^*^* & 48.426^\# & 494.0^\# \\
\hline
concerto.csv & 13.448^\# & 209.0^\# & 30.69^*^* & 74.0^*^* & 13.448^\# & 209.0^\# \\
\hline
sufia.csv & 0.0^\# & 6.0^\# & 0.0^*^* & 6.0^*^* & 0.0^\# & 6.0^\# \\
\hline
opal.csv & 10.268^\# & 132.0^\# & 20.089^*^* & 52.0^*^* & 10.268^\# & 132.0^\# \\
\hline
gradle.csv & 8.259^\# & 318.0^\# & 15.619^*^* & 138.0^*^* & 8.259^\# & 318.0^\# \\
\hline
rubinius.csv & 32.727^\# & 221.0^\# & 41.818^*^* & 105.0^*^* & 32.727^\# & 221.0^\# \\
\hline
graylog2-server.csv & 5.985^\# & 156.0^\# & 5.486^*^* & 58.0^*^* & 5.985^\# & 156.0^\# \\
\hline
metasploit-fram

In [108]:
ml = pd.read_csv('ml_pivot_result.csv')

In [112]:
projects = ml['project'].tolist()
col1 = ml['NB_builds'].tolist()
col2 = ml['NB_delay'].tolist()
nb_batch = ml['NB_Batch'].tolist()
col3 = ml['B4_builds'].tolist()
col4 = ml['B4_delay'].tolist()
b4_batch = ml['B4_Batch'].tolist()
col5 = ml['BB_builds'].tolist()
col6 = ml['BB_delay'].tolist()
bb_batch = ml['BB_Batch'].tolist()
col7 = ml['BS4_builds'].tolist()
col8 = ml['BS4_delay'].tolist()
bs4_batch = ml['BS4_Batch'].tolist()

In [114]:
for i in range(len(projects)):
    print("{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} \\\\".format(projects[i], 
                                                                round(col1[i], 3), batch_reps[nb_batch[i]], round(col2[i], 3), batch_reps[nb_batch[i]], 
                                                                round(col3[i], 3), batch_reps[b4_batch[i]], round(col4[i], 3), batch_reps[b4_batch[i]], 
                                                                round(col5[i], 3), batch_reps[bb_batch[i]], round(col6[i], 3), batch_reps[bb_batch[i]], 
                                                                round(col7[i], 3), batch_reps[bs4_batch[i]], round(col8[i], 3), batch_reps[bs4_batch[i]] ))
    print("\hline")

rails.csv & 43.399^*^* & 641.0^*^* & 58.158^\# & 2071.0^\# & 59.377^*^* & 651.0^*^* & 58.158^\# & 2071.0^\# \\
\hline
jruby.csv & 45.036^*^* & 186.0^*^* & 77.724^\# & 588.0^\# & 70.944^*^* & 194.0^*^* & 77.724^\# & 588.0^\# \\
\hline
cloud_controller_ng.csv & 42.688^*^* & 108.0^*^* & 65.217^\# & 351.0^\# & 57.312^*^* & 119.0^*^* & 65.217^\# & 351.0^\# \\
\hline
geoserver.csv & 44.643^*^* & 78.0^*^* & 98.81^\# & 252.0^\# & 83.929^*^* & 79.0^*^* & 98.81^\# & 252.0^\# \\
\hline
orbeon-forms.csv & 45.494^*^* & 106.0^*^* & 54.936^\# & 336.0^\# & 62.661^*^* & 106.0^*^* & 54.936^\# & 336.0^\# \\
\hline
sufia.csv & 26.316^\# & 30.0^\# & 47.368^\# & 30.0^\# & 52.632^*^* & 8.0^*^* & 47.368^\# & 30.0^\# \\
\hline
loomio.csv & 44.39^*^* & 91.0^*^* & 69.268^\# & 300.0^\# & 68.293^*^* & 97.0^*^* & 69.268^\# & 300.0^\# \\
\hline
puppet.csv & 37.681^*^* & 26.0^*^* & 26.087^\# & 86.0^\# & 39.13^*^* & 27.0^*^* & 26.087^\# & 86.0^\# \\
\hline
fog.csv & 44.444^*^* & 158.0^*^* & 67.236^\# & 507.0^\# & 68.3